In [1]:
import ipywidgets as widgets
from autocalc.autocalc import Var
from IPython.display import display, clear_output

# Packages for testing
#import random
#import functools

# Soft Tissue Sarcoma Mortality Predictor

#### Based on the paper of Yeramosu et al. (Submitted)
#### This model predicts whether a patient with soft-tissue sarcoma will experience cancer-related mortality (1) or not (0) and is to be used primarily by orthopaedic oncologists.
#### This is not a definitive outcome. All literature should be consulted.

In [11]:
# These are the input fields we receive (in order that they are input into the random forest model)

# The first parameter is the name, which is arbitrary and will only be used for display purposes
# It is optional to add a widget to a Var, but this makes it possible to use them as either input or output variables
v1 = Var('Chemotherapy?', initial_value = 0, widget = widgets.Dropdown(options=[('Yes', 1), ('No', 0)],value=0))
v2 = Var('Tumor Grade', initial_value = 0, widget = widgets.Dropdown(options=[('High', 1), ('Low', 0)],value=0))
v3 = Var('Histological Subtype', initial_value = 0, widget = widgets.Dropdown(options=[('Clear Cell Sarcoma', 0), \
        ('Epithelioid Sarcoma', 1), ('Leiomyosarcoma', 2), ('Liposarcoma', 3), \
        ('Malignant Peripheral Nerve Sheath Tumor', 4), ('Myxofibrosarcoma', 5), ('Other', 6), \
        ('Synovial Sarcoma', 7), ('Undifferentiated Pleomorphic Sarcoma', 8)] \
        ,value=0))
v4 = Var('M Stage', initial_value = 0, widget = widgets.Dropdown(options=[ ('0', 0), ('1', 1)],value=0))
v5 = Var('N Stage', initial_value = 0, widget = widgets.Dropdown(options=[ ('0', 0), ('1', 1)],value=0))
v6 = Var('Race', initial_value = 0, widget = widgets.Dropdown(options=[('White', 0), ('Black', 1), ('Other', 2)],value=0))
v7 = Var('Age', initial_value = 0, widget = widgets.Dropdown(options=[('0 years', 0), \
        ('1-4 years', 1), ('5-9 years', 5), ('10-14 years', 10), ('15-19 years', 15),\
        ('20-24 years', 20), ('25-29 years', 25), ('30-34 years', 30), ('35-39 years', 35),\
        ('40-44 years', 40), ('45-49 years', 45), ('50-54 years', 50), ('55-59 years', 55),\
        ('60-64 years', 60), ('65-69 years', 65), ('70-74 years', 70), ('75-79 years', 75),\
        ('80-84 years', 80), ('85+ years', 85)]\
        ,value=0))
v8 = Var('Sex', initial_value = 0, widget = widgets.Dropdown(options=[('Male', 0), ('Female', 1)],value=0))
v9 = Var('Tumor Location', initial_value = 1, widget = widgets.Dropdown(options=[('Upper Extremity', 1), ('Lower Extremity', 2), ('Trunk', 3)],value=1))
v10 = Var('Surgery?', initial_value = 0, widget = widgets.Dropdown(options=[('Yes', 1), ('No', 0)],value=0))
v11 = Var('T Stage', initial_value = 0, widget = widgets.Dropdown(options=[ ('0', 0), ('1', 1), ('2', 2), ('3', 3), ('4', 4)],value=0))
v12 = Var('Tumor Size (mm)', initial_value = 0, widget = widgets.BoundedFloatText(min=0))

v13 = Var('Radiation Therapy?', initial_value = 0, widget = widgets.Dropdown(options=[('Yes', 1), ('No', 0)],value=0))


## Demographic Info

In [4]:
display(v7)
display(v8)
display(v6)

## Tumor Characteristics

In [5]:
display(v2)
display(v12)
display(v11)
display(v5)
display(v4)
display(v3)
display(v9)

## Treatment Characteristics

In [14]:
display(v10)
display(v13)
display(v1)


# Prediction of Sarcoma Mortality

In [8]:
import pickle
# load the model from disk
filename = "finalized_RF_model.pkl"
rf_model = pickle.load(open(filename, 'rb'))

In [9]:
import numpy as np

In [15]:
# These are the inputs pulled from the user input

def getInput():
#     return {
#             u"Chemotherapy (1=Yes)": v1.get(),
#             u"Grade (4=anaplastic)": v2.get(),
#             u"Histology (See pic 12/19)": v3.get(),
#             u"M Stage": v4.get(),
#             u"N Stage": v5.get(),
#             u"Race (0=W, 1=B, 2=O, 3=U)": v6.get(),
#             u"Raw Age 2": v7.get(),
#             u"Sex (0=M)": v8.get(),
#             u"Site (1=upper extremity; 2=lower extremity/pelvis; 3=trunk)": v9.get(),
#             u"Surgery (1=Yes)": v10.get(),
#             u"T Stage": v11.get(),
#             u"Tumor Size": v12.get()
#     }
    return np.array([v7.get(), v6.get(), v8.get(), v9.get(), v2.get(), v3.get(), v11.get(), v5.get(), v4.get(),\
                              v10.get(), v13.get(), v1.get(), v12.get()]).reshape(1, -1)

#Metadata used in the model, only used for making the function call
tejaOutput = {
        u"Prob(Cause of Death (1=due to cancer)==0)": 0,
        u"Prob(Cause of Death (1=due to cancer)==1)": 0,
        u"Most Likely Cause of Death (1=due to cancer)": 0
}

button = widgets.Button(
    description='Calculate',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Calculate likelihood of cancer death',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

display(button)

i = Var('input', initial_value = getInput())
o = Var('Tumor Size', initial_value = tejaOutput)

# Old method of testing using autocalc -> ipywidgets output widget now utilized
#output = Var('Cancer Death Likely', fun=teja.score, inputs=[i, o], widget = widgets.FloatText(), read_only=True)
#output = Var('Cancer Death Likely', initial_value = random.randint(1, 10), widget = widgets.FloatText(), read_only=True)

out = widgets.Output()
display(out)

def on_button_clicked(b):
    with out:
        clear_output()
        i = Var('input', initial_value = getInput())
        #val = teja.score(getInput(), tejaOutput)
        output = Var('Mortality Likelihood', fun=rf_model.predict, inputs=[i], widget = widgets.FloatText(), read_only=True)
        display(widgets.VBox([]), output)
                     
button
button.on_click(on_button_clicked)

Button(button_style='success', description='Calculate', icon='check', style=ButtonStyle(), tooltip='Calculate …

Output()